<h1> QuantumSim - Shor's Nine Qubit Code </h1>
Author: Michel Meulen<br>
Date: 13-11-2024<br>
<br>
In this chapter an introduction is presented to quantum error correction (QEC) with an analysis of Peter Shors nine qubit code circuit. All content presented in this notebook is based of my literature study, which can be found: INSERT LINK. For running this notebook we use two libraries: quantumsim, which you are probably quite familiar with already, and quantumerrorcorrection, which contains circuits to create Shors nine qubit code and custom error gates.

In [1]:
from quantumsim import *
from quantumerrorcorrection import Shors

<h1> Quantum logic gates used in this notebook </h1>

<h3> NOT gate </h3>
<img src="./pictures/not_gate_white_background.png" alt="NOT quantum logic gate" style="width:15%;"/>
<p> NOT description </p>


<h3> CNOT gate </h3>
<img src="./pictures/cnot_gate_white_background.png" alt="CNOT quantum logic gate" style="width:15%;"/>
<p> CNOT description </p>

<h3> Toffoli gate (CCNOT) </h3>
<img src="./pictures/toffoli_gate_white_background.png" alt="Toffoli quantum logic gate" style="width:15%;"/>
<p> Toffoli description </p>

<h3> Hadamard gate </h3>
<img src="./pictures/hadamard_gate_white_background.png" alt="Hadamard quantum logic gate" style="width:15%;"/>

<h3> Double Hadamard gate </h3>
<img src="./pictures/hadamard_gate_white_background.png" alt="Hadamard quantum logic gate" style="width:10%;"/>
<img src="./pictures/hadamard_gate_white_background.png" alt="Hadamard quantum logic gate" style="width:10%;"/>

<h1> Bit-flip Correction Code </h1>
<img src="./pictures/bitflip_correction_circuit_white_background.png" alt="Bit flip correction circuit" style="width:50%;"/>

In [2]:
# Encode the logical qubit state among multiple qubits
bitFlipCorrectionCircuit = Circuit(3)
bitFlipCorrectionCircuit.pauli_x(0)
print("Initial value is |1>")
# Make init state: |100>

bitFlipCorrectionCircuit.cnot(0, 1)
bitFlipCorrectionCircuit.cnot(0, 2)
# |111>

# Create bitflip error
bitFlipCorrectionCircuit.bitflip_error_random()

# Decode, mayority check for correction
bitFlipCorrectionCircuit.cnot(0, 1)
bitFlipCorrectionCircuit.cnot(0, 2)
bitFlipCorrectionCircuit.toffoli(1, 2, 0)

bitFlipCorrectionCircuit.print_gates_and_descriptions()
bitFlipCorrectionCircuit.execute(print_state=False)
bitFlipCorrectionCircuit.measure(print_state=True)
# Measured answer must always be |1??>
print(bitFlipCorrectionCircuit.get_classical_state_of_qubit_as_string(0))


Initial value is |1>
X..	Pauli X on qubit 0
*X.	CNOT with control qubit 0 and target qubit 1
*.X	CNOT with control qubit 0 and target qubit 2
..X	Bit-flip error (Pauli X) on qubit 2
*X.	CNOT with control qubit 0 and target qubit 1
*.X	CNOT with control qubit 0 and target qubit 2
x**	Toffoli with control qubit 1 and CNOT with control qubit 2 and target qubit 0
Measured state:
|101>
|1>	 Measured value of qubit 0


## Phase-flip Correction Code
<img src="./pictures/phaseflip_correction_circuit_white_background.png" alt="Phase flip correction circuit" style="width:50%;"/>

In [3]:
# Encode the logical qubit state among multiple qubits
phaseFlipCorrectionCircuit = Circuit(3)
phaseFlipCorrectionCircuit.pauli_x(0)
print("Initial value is |1>")
# Make init state: |100>

phaseFlipCorrectionCircuit.cnot(0, 1)
phaseFlipCorrectionCircuit.cnot(0, 2)
# |111>

phaseFlipCorrectionCircuit.hadamard(0)
phaseFlipCorrectionCircuit.hadamard(1)
phaseFlipCorrectionCircuit.hadamard(2)

# Create Phase-flip error
phaseFlipCorrectionCircuit.phaseflip_error_random()

phaseFlipCorrectionCircuit.hadamard(0)
phaseFlipCorrectionCircuit.hadamard(1)
phaseFlipCorrectionCircuit.hadamard(2)

# Decode, mayority check for correction
phaseFlipCorrectionCircuit.cnot(0, 1)
phaseFlipCorrectionCircuit.cnot(0, 2)
phaseFlipCorrectionCircuit.toffoli(1, 2, 0)

phaseFlipCorrectionCircuit.print_gates_and_descriptions()
phaseFlipCorrectionCircuit.execute(print_state=False)
phaseFlipCorrectionCircuit.measure(print_state=True)

print(phaseFlipCorrectionCircuit.get_classical_state_of_qubit_as_string(0))


Initial value is |1>
X..	Pauli X on qubit 0
*X.	CNOT with control qubit 0 and target qubit 1
*.X	CNOT with control qubit 0 and target qubit 2
H..	Hadamard on qubit 0
.H.	Hadamard on qubit 1
..H	Hadamard on qubit 2
Z..	Phase-flip error (Pauli Z) on qubit 0
H..	Hadamard on qubit 0
.H.	Hadamard on qubit 1
..H	Hadamard on qubit 2
*X.	CNOT with control qubit 0 and target qubit 1
*.X	CNOT with control qubit 0 and target qubit 2
x**	Toffoli with control qubit 1 and CNOT with control qubit 2 and target qubit 0
Measured state:
|111>
|1>	 Measured value of qubit 0


## Shor's Nine Qubit Code
<img src="./pictures/shors_ninequbit_circuit_white_background.png" alt="Shors nine qubit code" style="width:50%;"/>

In [4]:
# Create the encoder
encoder = Shors.GetPhaseEncoderCircuit()
encoder.append_circuit(Shors.GetBitEncoderCircuit(0))
encoder.append_circuit(Shors.GetBitEncoderCircuit(3))
encoder.append_circuit(Shors.GetBitEncoderCircuit(6))

# Generate one random bit flip & phase flip error
flips = Shors.GetPhaseFlipErrorCircuit(0)
flips.append_circuit(Shors.GetBitFlipErrorCircuit(0))

# Decode 
decoder = Shors.GetBitDecoderCircuit(0)
decoder.append_circuit(Shors.GetBitDecoderCircuit(3))
decoder.append_circuit(Shors.GetBitDecoderCircuit(6))
decoder.append_circuit(Shors.GetPhaseDecoderCircuit())

# Glue all parts of the circuit together
shorsCode = Circuit(9)
print("Initial value is |0>")
shorsCode.append_circuit(encoder)
shorsCode.append_circuit(flips)
shorsCode.append_circuit(decoder)

# Execute and measure the ShorsCode
shorsCode.print_gates_and_descriptions()
shorsCode.execute(print_state=False)
shorsCode.measure(print_state=True)

print(shorsCode.get_classical_state_of_qubit_as_string(0))

Initial value is |0>
*..X.....	CNOT with control qubit 0 and target qubit 3
*.....X..	CNOT with control qubit 0 and target qubit 6
H........	Hadamard on qubit 0
...H.....	Hadamard on qubit 3
......H..	Hadamard on qubit 6
*X.......	CNOT with control qubit 0 and target qubit 1
*.X......	CNOT with control qubit 0 and target qubit 2
...*X....	CNOT with control qubit 3 and target qubit 4
...*.X...	CNOT with control qubit 3 and target qubit 5
......*X.	CNOT with control qubit 6 and target qubit 7
......*.X	CNOT with control qubit 6 and target qubit 8
.......Z.	Phase-flip error (Pauli Z) on qubit 7
...X.....	Bit-flip error (Pauli X) on qubit 3
*X.......	CNOT with control qubit 0 and target qubit 1
*.X......	CNOT with control qubit 0 and target qubit 2
x**......	Toffoli with control qubit 1 and CNOT with control qubit 2 and target qubit 0
...*X....	CNOT with control qubit 3 and target qubit 4
...*.X...	CNOT with control qubit 3 and target qubit 5
...x**...	Toffoli with control qubit 4 and CNOT